<a href="https://colab.research.google.com/github/sdam-au/mounds_ETL/blob/master/scripts/EXPLORING-MOUNDS-IN-GSHEETS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

### GENERAL REQUIREMENTS - will install the libraries
import numpy as np
import math
import pandas as pd
from google.colab import files
import sys
import requests
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import io
# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import warnings
warnings.filterwarnings('ignore')
from functools import reduce


import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
from oauth2client.client import GoogleCredentials

!pip install sddk ### our own package under construction, always install to have an up-to-date version
import sddk

In [2]:
### configure session and groupurl
### in the case of "SDAM_root", the folder owner is Vojtech with username 648597@au.dk
conf = sddk.configure_session_and_url("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


# Reading and merging the data

In [162]:
### authorize google sheets 
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

# establish connection with particular sheets by their url:
AORDataset = gc.open_by_url("https://docs.google.com/spreadsheets/d/1cx0nntcCLgrwQvCvvIYkjFJ-TnoJ0QRJfTgrf2qEEv4/edit?usp=sharing")
IzvestiaDataset = gc.open_by_url("https://docs.google.com/spreadsheets/d/1wOxbKVHGNHox4h86Z5ZXXADUubMJofC1KqDLMLT13a8/edit?usp=sharing")

# Preprocessing AOR dataset

In [ ]:
# what worksheets we have:
AORDataset.worksheets()

[<Worksheet 'GeneralSpatial' id:1795672216>,
 <Worksheet 'MoundAttributes' id:819285652>,
 <Worksheet 'BurialAttributes' id:1240910271>,
 <Worksheet 'Lookups' id:504663605>,
 <Worksheet 'MoundNumbers' id:2134988134>,
 <Worksheet 'AOR_aggr' id:814313151>]

In [ ]:
# importing three lists into three dataframes
AORDataset_GeneralSpatial = pd.DataFrame(AORDataset.worksheet("GeneralSpatial").get_all_records())
AORDataset_MoundAttributes = pd.DataFrame(AORDataset.worksheet("MoundAttributes").get_all_records())
AORDataset_BurialAttributes = pd.DataFrame(AORDataset.worksheet("BurialAttributes").get_all_records())

In [ ]:
AORDataset_BurialAttributes.head(5)

,MoundID,Name,GraveNo,Enclosure Type,LaborAssessment,Burial No,Extra skeletal remains?,Notes on burial,Assemblage?,AnimalRemains,Lithics,HMPottery,Storage,FineVessels,DrinkingCups,Jewellery,Weapons,SpecialFinds,Imports,Grave Rank symbols,StartDate,Enddate,Chronology rating
0,1002,Mound 5,1,Simple pit,no elaboration (eg.a pit or no indistinct encl...,1,N,,N,,,,,,,,,,,0 - No symbols of status present,-3500,-3000,1 - rough estimate by author on basis of the a...
1,1002,Mound 5,1,Simple pit,no elaboration (eg.a pit or no indistinct encl...,2,N,,N,,,,,,,,,,,0 - No symbols of status present,-3500,-3000,1 - rough estimate by author on basis of the a...
2,1002,Mound 5,2,Simple pit,no elaboration (eg.a pit or no indistinct encl...,1,N,,N,,,,,,,,,,,0 - No symbols of status present,-3500,-3000,1 - rough estimate by author on basis of the a...
3,1002,Mound 5,2,Simple pit,no elaboration (eg.a pit or no indistinct encl...,2,N,,N,,,,,,,,,,,0 - No symbols of status present,-3500,-3000,1 - rough estimate by author on basis of the a...
4,1003,No name,1,Simple pit,no elaboration (eg.a pit or no indistinct encl...,1,N,Main Burial (earlier),Y,,Y (2),,,,,2(silver spirals),,red ochre,,"1 - One or two status symbols (ochre, personal...",-3500,-3000,3 - a number of burial aspects consulted and c...


In [ ]:
# explore unique values within "StartDate" and "Enddate"
# (to avoid unintentional remove of valuable info like "beginning" or "end")
print(AORDataset_BurialAttributes["Enddate"].unique().tolist())

[-3000, 200, 225, 250, 400, 150, 500, -600, 300, -50, -280, '', -200, 125, 1300, 100, -1100, -375, -250, 25, -300, 1100, 75, -2700, -1600, -1800, -2000, 1400, 1700, 50, 1800, -350, 240, '?', 1200, -275, 0, 1850, -2500, -3200, -800, -1200, -700, -325, -290, 'no data', 180, 120, -1700, -2800, -2200, 175, 1600, 325, -1000, -900, 1500, -420, -400, -650, -2100, 1000, -500, 350, -1350, 425, -100, -1150, 600, -450, 1878, 450, 170, 1050, 1900, 1825, -425, -475, -2600, 315, -285, -75, 375, -850, 275, -575, 320, -150, 1250, -550]


In [ ]:
# force column to integers
def get_int(value):
  try:
      variable = int(value)
  except ValueError:
      variable = None
  return variable
for column in ["StartDate", "Enddate"]:
  AORDataset_BurialAttributes[column] = AORDataset_BurialAttributes.apply(lambda row: get_int(row[column]), axis=1)

In [5]:
# grouping BurialAttributes by mounds - function

def get_data_from_group(group):
  """
  function to get data from a group class object produced by pandas groupby() function
  """
  group = pd.DataFrame(group).reset_index() # transform group object into dataframe object
  group.sort_values("StartDate", inplace=True) # sort it in by "StartData" column (ascending order)
  mound = {"MoundID" : group["MoundID"][0]} # get mound ID (on the basis of the first row, but should be the same for all rows within a group)
  mound["Graves"] = len(group) # how many rows we have 
  mound["FirstStartDate"] = group['StartDate'][0] # get "StartDate" from the first row
  mound["FirstEndDate"] = group['Enddate'][0]  # get "Enddate" from the first row
  mound["MaxEndDate"] = group["Enddate"].max() # get maximum value for "Enddate" for all rows
  mound["FirstChronologyRating"] = group["Chronology rating"][0]
  mound["FirstEnclosureType"] = group["Enclosure Type"][0]
  mound["FirstBurialNumber"] = group["Burial No"][0]
  mound["GraveRankSymbols"] = list(set([rank.partition(" -")[0] for rank in group["Grave Rank symbols"].tolist()])) # list of all unique
  mound["AllDates"] = [(date[0], date[1]) for date in zip(group['StartDate'], group['Enddate'])] # list of tuples of all dates 
  try:
    mound["TotalBurialNumber"] = group["Burial No"].astype(int).sum() # total number of burials associated with given mound
  except:
    mound["TotalBurialNumber"] = None
  mound["BurialAttributesAggr"] = pd.DataFrame(group).reset_index().T.to_dict() # all data packed from given group packed into a dictionary, rows as keys
  return mound

In [ ]:
### apply the function
AORDataset_BurialAttributes_grouped = pd.DataFrame(AORDataset_BurialAttributes.sort_values("StartDate").groupby("MoundID").apply(lambda group_df: get_data_from_group(group_df)).tolist())
AORDataset_BurialAttributes_grouped.head(10)

In [ ]:
# example of one value nested within "BurialAttributesAggr" column
AORDataset_BurialAttributes_grouped[AORDataset_BurialAttributes_grouped["MoundID"]== 1115]["BurialAttributesAggr"].tolist()[0][6]

{'AnimalRemains': '',
 'Assemblage?': 'Y',
 'Burial No': '',
 'Chronology rating': '1 - rough estimate by author on basis of the assemblage',
 'DrinkingCups': '',
 'Enclosure Type': 'Funeral pyre in a pit',
 'Enddate': 250.0,
 'Extra skeletal remains?': '',
 'FineVessels': '',
 'Grave Rank symbols': 'not available',
 'GraveNo': 11,
 'HMPottery': '',
 'Imports': '',
 'Jewellery': '',
 'LaborAssessment': 'some elaboration (marks of a coffin, small stone circle, tiled or stone lined grave of RM period)',
 'Lithics': '',
 'MoundID': 1115,
 'Name': 'Mound 2',
 'Notes on burial': '',
 'SpecialFinds': '',
 'StartDate': 150.0,
 'Storage': '',
 'Weapons': '',
 'index': 828,
 'level_0': 10}

In [ ]:
# exporting to gsheet
#set_with_dataframe(AORDataset.add_worksheet("BurialAttributes_grouped", 1, 1), AORDataset_BurialAttributes_grouped)

In [ ]:
# merge lists using "MoundID"
dfs = [AORDataset_GeneralSpatial, AORDataset_MoundAttributes, AORDataset_BurialAttributes_grouped]
AOR_df = dfs[0].merge(dfs[1], on="MoundID", how="inner").merge(dfs[2], on="MoundID", how="inner")

In [ ]:
AOR_df.head(5)

,MoundID,Municipality,Region,Lat,Long,Error radius(m),LU_Around,MoundCover,Geomorphology,Prominence,MoundName,HeightMax,HeightMin,DiameterMax,DiameterMin,GT,RT,Condition,Source of Impact,NumberOfBurialsInside,Graves,FirstStartDate,FirstEndDate,MaxEndDate,FirstChronologyRating,FirstEnclosureType,FirstBurialNumber,GraveRankSymbols,AllDates,TotalBurialNumber,BurialAttributesAggr
0,1107,Sliven,Sliven,42.4916,26.2698,0,Annual agriculture,No data,hillside,"somewhat prominent (on a plateau, slope or spu...",no name,5.6,1.6,36,34,,Y,2 - partially damaged (type of damage is descr...,looting,10,10,0.0,100.0,400.0,2 - specialist consulted on a single item,,1,"[0, 1]","[(0.0, 100.0), (0.0, 100.0), (0.0, 100.0), (0....",10.0,"{0: {'level_0': 0, 'index': 743, 'MoundID': 11..."
1,1179,Belitsa,Blagoevgrad,41.9357,23.5736,0,Pasture (grassland),Pasture (grassland),on the ridge,very (on an outcrop or peak),Mogilata sas sarkofaga,7,,30,,,N,"1 - pristine, (only superficial or minor damag...",,1,1,-400.0,-300.0,-300.0,2 - specialist consulted on a single item,Cist grave,1,[2],"[(-400.0, -300.0)]",1.0,"{0: {'level_0': 0, 'index': 1190, 'MoundID': 1..."
2,1392,Belitsa,Blagoevgrad,41.9198,23.5723,0,No data,No data,on the ridge,"somewhat prominent (on a plateau, slope or spu...",Golyamata mogila,4,,30,25,,N,"1 - pristine, (only superficial or minor damag...",,1,1,NaN,NaN,NaN,0 - no chronology,Simple pit,1,[0],"[(nan, nan)]",1.0,"{0: {'level_0': 0, 'index': 1992, 'MoundID': 1..."
3,1393,Blagoevgrad,Blagoevgrad,41.9769,23.0931,100,Scrub,No data,hilltop,very (on an outcrop or peak),Kaymenska chuka,,,,,,Y,2 - partially damaged (type of damage is descr...,looting,4,4,200.0,300.0,300.0,1 - rough estimate by author on basis of the a...,Tiled grave,1,[0],"[(200.0, 300.0), (200.0, 300.0), (200.0, 300.0...",4.0,"{0: {'level_0': 0, 'index': 1993, 'MoundID': 1..."
4,1012,Primorsko,Burgas,42.2778,27.7269,200,Forest,Forest,"valley bottom, or flat ground","not prominent (in a valley, plain or depression)",Chenger 1,3.25,2.98,28.6,26.8,,N,"1 - pristine, (only superficial or minor damag...",,2,2,-400.0,-200.0,-200.0,1 - rough estimate by author on basis of the a...,Simple pit,1,"[2, 1]","[(-400.0, -200.0), (nan, nan)]",2.0,"{0: {'level_0': 0, 'index': 120, 'MoundID': 10..."


In [ ]:
# export to gsheet
# (without BurialAttributesAggr column)
set_with_dataframe(AORDataset.add_worksheet("AOR_aggr", 1, 1), AOR_df.drop("BurialAttributesAggr", axis=1))

In [ ]:
# export to sciencedata
sddk.write_file("SDAM_data/mounds/AOR_df_2020-06-24.json", AOR_df, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/mounds/AOR_df_2020-06-24.json"


# Preprocessing Izvestia dataset (cleaning datations)

In [25]:
# what worksheets we have:
IzvestiaDataset.worksheets()

[<Worksheet 'General' id:369527838>,
 <Worksheet 'MoundAttributes' id:548739213>,
 <Worksheet 'BurialAttributes' id:18001881>,
 <Worksheet 'FlatCemeteries' id:34049049>,
 <Worksheet 'TimeOnTask' id:136710821>,
 <Worksheet 'Lookups' id:935910602>,
 <Worksheet 'Izvestia_aggr' id:898354309>,
 <Worksheet 'dates_strings' id:1173555538>,
 <Worksheet 'phases' id:1394002736>,
 <Worksheet 'centuries' id:1885898853>]

In [164]:
# importing three lists into three dataframes
IzvestiaDataset_General = pd.DataFrame(IzvestiaDataset.worksheet("General").get_all_records())
IzvestiaDataset_MoundAttributes = pd.DataFrame(IzvestiaDataset.worksheet("MoundAttributes").get_all_records())
IzvestiaDataset_BurialAttributes = pd.DataFrame(IzvestiaDataset.worksheet("BurialAttributes").get_all_records())

In [165]:
IzvestiaDataset_BurialAttributes["StartDate"].unique().tolist()

[-450,
 -200,
 'N/A',
 -600,
 200,
 100,
 ' possible data-201',
 ' possible data-200',
 300,
 'the end -500',
 'end of -600',
 -500,
 -400,
 'second half of 100',
 'second half of -500',
 'end of 200',
 -300,
 500,
 'end of -400',
 'second half of 400',
 'end of -500',
 'end of -200',
 'middle of 200',
 'end of 100',
 '',
 'end of 099',
 'end of 101',
 'end of 102',
 'end of -300',
 -700,
 -1100,
 'end of -700']

In [167]:
# defining certainty column
def get_certainty(row):
  certainty = ""
  for column in ["StartDate", "Enddate"]:
    if "possible data" in str(row[column]):
      certainty = certainty + "?"
  return certainty
IzvestiaDataset_BurialAttributes["DateCertainty"] = IzvestiaDataset_BurialAttributes.apply(lambda row: get_certainty(row), axis=1)

In [168]:
IzvestiaDataset_BurialAttributes[IzvestiaDataset_BurialAttributes["DateCertainty"].str.startswith("?")].head(10)

,MoundID,Name,GraveNo,Enclosure Type,Enclosure dimensions,Enclosure materials,LaborAssessment,Burial No,Burial type,Sex,Age,SkeletonOrder,SkeletonPosition,InhumOrientation,Extra skeletal remains?,Notes on burial,Assemblage?,Lithics,CoarsePottery,Storage,FineVessels,DrinkingCups,Jewellery,Weapons,SpecialFinds,Imports,Grave Rank symbols,Notes of burial assemblage,StartDate,Enddate,Date based on,ChronoResource,Chronology rating,Reflections on chronology,DateCertainty
90,44,Paiydushko mound 3,1,Simple pit,d=1.05-1.10 m; depth:0.50 m.,pit,no elaboration (eg.a pit or no indistinct encl...,1,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,few pieces of a grey colored vessel,,,,,,,N,0 - No symbols of status present,,possible data-201,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
91,44,Paiydushko mound 3,2,Simple pit,d=0.85-1.00 m; depth:0.25 m.,pit,no elaboration (eg.a pit or no indistinct encl...,2,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
92,44,Paiydushko mound 3,3,Simple pit,d=1.20-1.40 m; depth:0.40 m.,pit,no elaboration (eg.a pit or no indistinct encl...,3,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
93,44,Paiydushko mound 3,4,Simple pit,d=1.20-1.30 m; depth:0.40 m.,pit,no elaboration (eg.a pit or no indistinct encl...,4,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
94,44,Paiydushko mound 3,5,Simple pit,0.90x0.75 m/ 0.80x0.60 m. Depth:0.35 m.,pit,no elaboration (eg.a pit or no indistinct encl...,5,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
95,44,Paiydushko mound 3,6(hearth),none,,hearth,no elaboration (eg.a pit or no indistinct encl...,6,not available,not available,not available,not available,not available,not available,N,The hearht were found on the end of a robber t...,N,N,,pieces of a bottom of amphora,,,,,iron object,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
96,45,Paiydushko mound 4,1 (hearht),none,5.80x2.70 m.,hearth,not available,1,not available,not available,not available,not available,not available,not available,N,The hearht were found at the center of the mou...,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,36",1 - rough estimate by author on basis of the a...,picture available,?
97,45,Paiydushko mound 4,2,Simple pit,d=1.06-1.20 m; depth:0.40 m.,pit,not available,2,not available,not available,not available,not available,not available,not available,N,The hearht were found at the center of the mou...,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,36",1 - rough estimate by author on basis of the a...,picture available,?
98,45,Paiydushko mound 4,3,Simple pit,d=0.95-1.10 m; depth:0.40 m.,pit,not available,3,not available,not available,not 

In [169]:
IzvestiaDataset_BurialAttributes[IzvestiaDataset_BurialAttributes["StartDate"].str.contains("[a-z]", na=False)]

,MoundID,Name,GraveNo,Enclosure Type,Enclosure dimensions,Enclosure materials,LaborAssessment,Burial No,Burial type,Sex,Age,SkeletonOrder,SkeletonPosition,InhumOrientation,Extra skeletal remains?,Notes on burial,Assemblage?,Lithics,CoarsePottery,Storage,FineVessels,DrinkingCups,Jewellery,Weapons,SpecialFinds,Imports,Grave Rank symbols,Notes of burial assemblage,StartDate,Enddate,Date based on,ChronoResource,Chronology rating,Reflections on chronology,DateCertainty
90,44,Paiydushko mound 3,1,Simple pit,d=1.05-1.10 m; depth:0.50 m.,pit,no elaboration (eg.a pit or no indistinct encl...,1,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,few pieces of a grey colored vessel,,,,,,,N,0 - No symbols of status present,,possible data-201,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
91,44,Paiydushko mound 3,2,Simple pit,d=0.85-1.00 m; depth:0.25 m.,pit,no elaboration (eg.a pit or no indistinct encl...,2,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
92,44,Paiydushko mound 3,3,Simple pit,d=1.20-1.40 m; depth:0.40 m.,pit,no elaboration (eg.a pit or no indistinct encl...,3,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
93,44,Paiydushko mound 3,4,Simple pit,d=1.20-1.30 m; depth:0.40 m.,pit,no elaboration (eg.a pit or no indistinct encl...,4,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
94,44,Paiydushko mound 3,5,Simple pit,0.90x0.75 m/ 0.80x0.60 m. Depth:0.35 m.,pit,no elaboration (eg.a pit or no indistinct encl...,5,not available,not available,not available,not available,not available,not available,N,No inventar and bones found.,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
95,44,Paiydushko mound 3,6(hearth),none,,hearth,no elaboration (eg.a pit or no indistinct encl...,6,not available,not available,not available,not available,not available,not available,N,The hearht were found on the end of a robber t...,N,N,,pieces of a bottom of amphora,,,,,iron object,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,35",1 - rough estimate by author on basis of the a...,picture available,?
96,45,Paiydushko mound 4,1 (hearht),none,5.80x2.70 m.,hearth,not available,1,not available,not available,not available,not available,not available,not available,N,The hearht were found at the center of the mou...,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,36",1 - rough estimate by author on basis of the a...,picture available,?
97,45,Paiydushko mound 4,2,Simple pit,d=1.06-1.20 m; depth:0.40 m.,pit,not available,2,not available,not available,not available,not available,not available,not available,N,The hearht were found at the center of the mou...,N,N,,,,,,,,N,0 - No symbols of status present,,possible data-200,-100,as a part of the necropolis,"Ginev1995,36",1 - rough estimate by author on basis of the a...,picture available,?
98,45,Paiydushko mound 4,3,Simple pit,d=0.95-1.10 m; depth:0.40 m.,pit,not available,3,not available,not available,not 

In [170]:
centuries = pd.DataFrame(IzvestiaDataset.worksheet("centuries").get_all_records())
centuries_BC = centuries.set_index("start_BC").T.to_dict()
centuries_BC

{-800: {'arabic': '8th', 'start_AD': 701, 'stop_AD': 800, 'stop_BC': -701},
 -700: {'arabic': '7th', 'start_AD': 601, 'stop_AD': 700, 'stop_BC': -601},
 -600: {'arabic': '6th', 'start_AD': 501, 'stop_AD': 600, 'stop_BC': -501},
 -500: {'arabic': '5th', 'start_AD': 401, 'stop_AD': 500, 'stop_BC': -401},
 -400: {'arabic': '4th', 'start_AD': 301, 'stop_AD': 400, 'stop_BC': -301},
 -300: {'arabic': '3rd', 'start_AD': 201, 'stop_AD': 300, 'stop_BC': -201},
 -200: {'arabic': '2nd', 'start_AD': 101, 'stop_AD': 200, 'stop_BC': -101},
 -100: {'arabic': '1st', 'start_AD': 1, 'stop_AD': 100, 'stop_BC': -1}}

In [171]:
centuries = pd.DataFrame(IzvestiaDataset.worksheet("centuries").get_all_records())
centuries_AD = centuries.set_index("stop_AD").T.to_dict()
centuries_AD

{100: {'arabic': '1st', 'start_AD': 1, 'start_BC': -100, 'stop_BC': -1},
 200: {'arabic': '2nd', 'start_AD': 101, 'start_BC': -200, 'stop_BC': -101},
 300: {'arabic': '3rd', 'start_AD': 201, 'start_BC': -300, 'stop_BC': -201},
 400: {'arabic': '4th', 'start_AD': 301, 'start_BC': -400, 'stop_BC': -301},
 500: {'arabic': '5th', 'start_AD': 401, 'start_BC': -500, 'stop_BC': -401},
 600: {'arabic': '6th', 'start_AD': 501, 'start_BC': -600, 'stop_BC': -501},
 700: {'arabic': '7th', 'start_AD': 601, 'start_BC': -700, 'stop_BC': -601},
 800: {'arabic': '8th', 'start_AD': 701, 'start_BC': -800, 'stop_BC': -701}}

In [172]:
centuries_BC[-600]

{'arabic': '6th', 'start_AD': 501, 'stop_AD': 600, 'stop_BC': -501}

In [180]:
phases = [("beggining", 10), ("end", 90), ("half", 50)]

def modify_by_phase(datation):
  try:
    return int(datation)
  except:
    if isinstance(datation, str):
      #BC and AD require different approach, so:
      modifier = [phase[1] for phase in phases if phase[0] in datation]
      if len(modifier) > 0:
        for century in centuries_BC.keys():
          if str(century) in datation:
            new_date = century + modifier[0]
            return new_date
            break
        for century in centuries_AD.keys():
          if str(century) in datation:
            new_date = centuries_AD[century]["start_AD"] + modifier[0]
            return new_date
            break
      else:
        match = re.search("\-?\d+", datation)
        if match:
          new_date = int(match[0])
        else:
          new_date = None
        return new_date
    else:
      return None 

In [181]:
# testing
datations = ["beggining of 200", "second half of -400", "possible 150", -750]
for datation in datations:
  print(modify_by_phase(datation))

111
-350
150
-750


In [182]:
IzvestiaDataset_BurialAttributes["StartDate"] = IzvestiaDataset_BurialAttributes.apply(lambda row: modify_by_phase(row["StartDate"]), axis=1)
IzvestiaDataset_BurialAttributes["Enddate"] = IzvestiaDataset_BurialAttributes.apply(lambda row: modify_by_phase(row["Enddate"]), axis=1)

# Aggregating Izvestia dataset

In [183]:
IzvestiaDataset_BurialAttributes_grouped = pd.DataFrame(IzvestiaDataset_BurialAttributes.sort_values("StartDate").groupby("MoundID").apply(lambda group_df: get_data_from_group(group_df)).tolist())
IzvestiaDataset_BurialAttributes_grouped.head(10)

,MoundID,Graves,FirstStartDate,FirstEndDate,MaxEndDate,FirstChronologyRating,FirstEnclosureType,FirstBurialNumber,GraveRankSymbols,AllDates,TotalBurialNumber,BurialAttributesAggr
0,2,1,-450.0,-300.0,-300.0,1 - rough estimate by author on basis of the a...,Urn,1,[1],"[(-450.0, -300.0)]",1.0,"{0: {'level_0': 0, 'index': 0, 'MoundID': 2, '..."
1,3,2,-200.0,-300.0,-300.0,1 - rough estimate by author on basis of the a...,Urn,1,"[, 1]","[(-200.0, -300.0), (-200.0, -300.0)]",NaN,"{0: {'level_0': 0, 'index': 1, 'MoundID': 3, '..."
2,4,6,-450.0,-350.0,-350.0,1 - rough estimate by author on basis of the a...,Urn,1,"[0, ]","[(-450.0, -350.0), (nan, nan), (nan, nan), (na...",8.0,"{0: {'level_0': 0, 'index': 8, 'MoundID': 4, '..."
3,5,1,-450.0,-400.0,-400.0,1 - rough estimate by author on basis of the a...,not available,,[not available],"[(-450.0, -400.0)]",NaN,"{0: {'level_0': 0, 'index': 9, 'MoundID': 5, '..."
4,6,2,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,,2,"[0, 1]","[(-600.0, -450.0), (-600.0, -450.0)]",3.0,"{0: {'level_0': 0, 'index': 11, 'MoundID': 6, ..."
5,7,2,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,,2,"[, AA]","[(-600.0, -450.0), (-600.0, -450.0)]",3.0,"{0: {'level_0': 0, 'index': 13, 'MoundID': 7, ..."
6,8,2,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,1,"[0, 2]","[(-600.0, -450.0), (-600.0, -450.0)]",3.0,"{0: {'level_0': 0, 'index': 14, 'MoundID': 8, ..."
7,9,4,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,1,"[0, 1]","[(-600.0, -450.0), (-600.0, -450.0), (-600.0, ...",10.0,"{0: {'level_0': 0, 'index': 16, 'MoundID': 9, ..."
8,10,3,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,1,"[0, 1]","[(-600.0, -450.0), (-600.0, -450.0), (-600.0, ...",6.0,"{0: {'level_0': 0, 'index': 20, 'MoundID': 10,..."
9,11,3,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,Tomb of brick or stone,1,"[0, 1]","[(-600.0, -450.0), (-600.0, -450.0), (-600.0, ...",6.0,"{0: {'level_0': 0, 'index': 23, 'MoundID': 11,..."


In [185]:
# merge using "MoundID"
dfs = [IzvestiaDataset_General, IzvestiaDataset_MoundAttributes, IzvestiaDataset_BurialAttributes_grouped]
Izvestia_df = dfs[0].merge(dfs[1], on="MoundID", how="inner").merge(dfs[2], on="MoundID", how="inner")

In [186]:
Izvestia_df

,MoundID,MoundName_x,Location description in text,Municipality,Region,Lat,Long,LocPrecision,Source,Error radius(m),LU_Around,MoundCover,Prominence,Reason for excavation,Current status,PhotoExists,DrawingExists,Asociated settlement,Distance to assoc.settlement (m),References,,BG Code,Mound Date,Bara Notes,MoundName_y,BiblioReference,Elev_msl,HeightMin,HeightMax,Dim1,Dim2,GT,Notes on Dimensions,RT,Condition,Source of Impact,NumberOfBurialsInside,Graves_x,HumanBurialsRemains,Cenotaph?,AnimalRemainsNoHuman,Notes on Mound Condition and Grave number,Graves_y,FirstStartDate,FirstEndDate,MaxEndDate,FirstChronologyRating,FirstEnclosureType,FirstBurialNumber,GraveRankSymbols,AllDates,TotalBurialNumber,BurialAttributesAggr
0,2,Divdyadovo m.,Diuzmeshe area,Divdyadovo,Shumen,N/A,N/A,not available,,the author did not specify the distance from ...,agricultural field,crops,"somewhat prominent (on a plateau, slope or spu...",Research - Large attractive mound,"Partially excavated, still stands",Y,Y,N,N/A,Dremsizova 1963,,,,I have no idea where is the given village and ...,Divdyadovo m.,"Dremsizova 1963, 2-3",N/A,N/A,1,10,12,,,N,2 - partially damaged (type of damage is descr...,ploughing,1,1,1,,,The number of graves is 1 cremation,1,-450.0,-300.0,-300.0,1 - rough estimate by author on basis of the a...,Urn,1,[1],"[(-450.0, -300.0)]",1.0,"{0: {'level_0': 0, 'index': 0, 'MoundID': 2, '..."
1,3,Malamir m.,"2 km west, next to the Malamir village",Malamir v.,Shumen,N/A,N/A,not available,,the author did not specify the distance from ...,agricultural field,crops,not available,,"Partially excavated, still stands",Y,Y,N,2000,,,,,I have no idea where is the given village and ...,Malamir m.,"Dremsizova 1963, 6-7",N/A,0.8,1,10,12,,,N,2 - partially damaged (type of damage is descr...,ploughing,2 (1 urn and 1 tomb),2,1,Y,,The number of graves is 1 cremation and one d...,2,-200.0,-300.0,-300.0,1 - rough estimate by author on basis of the a...,Urn,1,"[, 1]","[(-200.0, -300.0), (-200.0, -300.0)]",NaN,"{0: {'level_0': 0, 'index': 1, 'MoundID': 3, '..."
2,4,Kalugeritsa necropolis,Kanara bashi area,Kalugeritsa v.,Shumen,N/A,N/A,not available,,the author did not specify the distance from ...,agricultural field,crops,"not prominent (in a valley, plain or depression)",,"Partially excavated, and removed",Y,N,N,N/A,"Damyanov,Popov 1972",,,,I have no idea where is the given village and ...,Kalugeritsa necopolis,"Damyanov,Popov1972 ,81-94",N/A,N/A,N/A,N/A,N/A,,,N,5 - nearly extinct (archaeologically useful cu...,communications (road on or next to mound),4 graves (6 burials),4,6,,,"The number of graves is 4 , but there are 6 bu...",6,-450.0,-350.0,-350.0,1 - rough estimate by author on basis of the a...,Urn,1,"[0, ]","[(-450.0, -350.0), (nan, nan), (nan, nan), (na...",8.0,"{0: {'level_0': 0, 'index': 8, 'MoundID': 4, '..."
3,5,Dyadozlatyuvata m.,Goliyam Yurdan area,Smyadovo,Shumen,43.0516,26.9796,3 - legacy reports (very coarse),Topo 50,7500,agricultural field,crops,not available,,not available,Y,N,N,N/A,Dremsizova 1963,,,,I don't have the publication,Dyadozlatyuvata m.,"Dremsizova 1963, 8-10",N/A,N/A,N/A,N/A,N/A,,,N,2 - partially damaged (type of damage is descr...,ploughing,NA,NA,NA,,,,1,-450.0,-400.0,-400.0,1 - rough estimate by author on basis of the a...,not available,,[not available],"[(-450.0, -400.0)]",NaN,"{0: {'level_0': 0, 'index': 9, 'MoundID': 5, '..."
4,6,Mound 1,"Kurchan tepe area,Dobrina necropolis",Dobrina v.,Provadiya,43.2076,27.4598,3 - legacy reports (very coarse),"Topo 50 / Google Earth / Mirchev 1965, 34 - 35",150,stony field,thin black earth,"somewhat prominent (on a plateau, slope or spu...",Research - Large attractive mound,Excavated and removed,Y,N,Y(other 27 mounds of the necropolis),N/A,Mirchev 1965,,,,,Mound 1,"Mirchev1965, 35-36",N/A,0.15,0.2,7.5,N/A,,,N,not available,not available,1 grave (2 burials),1,2,,,"Small mound, cover with stones, and a couple o...",2,-600.0,-450.0,-450.0,3 - a number of burial aspects consulted and c...,,2,"[0, 1]","

In [187]:
# export to gsheet
set_with_dataframe(IzvestiaDataset.add_worksheet("Izvestia_aggr", 1,1), Izvestia_df.drop("BurialAttributesAggr", axis=1))

In [ ]:
sddk.write_file("SDAM_data/mounds/Izvestia_df.json", Izvestia_df, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/mounds/Izvestia_df.json"
